<h1> Pick and Place Example

In [1]:
import rmlib
import time

In [2]:
rm = rmlib.RMLib()

Robot Ready


### <u>Lets start by defining a waypoint. There are two ways of saving a waypoint: 
1. The first way is to save the pose of the tool-tip (TCP) which is stored as an array of [X, Y, Z, rX, rY, rZ]. 
<br>
2. The second way is to store the joint angles of the robot which is also stored as and array with [J1, J2, J3, J4, J5, J6] with Ji is the angle at joint i in radians for a 6-DoF robot.

In this example we will store our waypoints as joint angles because it eliminates the possibility of overrotating a joint. 
<br><br>
We will also save the camera transform. The camera transform is a transformation matrix describing the position and orientation of the camera with respect to the base of the robot. We will use the camera transform later to transform objects from a local frame (relative to the camera) to a global frame (relative to the robot base.)
<br>
#### <b>Position the robot in an orientation that it can see the objects

In [3]:
rm.align_gripper_with_z()
pickup_waypoint = rm.get_joint_angles()

# Normalize the wrist joint to give us the largest joint freedom when picking
pickup_waypoint = rm.normalize_wrist_angle(pickup_waypoint)

camera_transform = rm.get_camera_transform()

### <u>Next lets tune the disparity shift
The disparity shift is a camera setting that corrects for the parallax of the two camera lenses. The disparity shift is dependent on the mean height of the point cloud. If the disparity is not set correctly your cloud will be distorted. This particular tuning method sets the disparity based on the estimated distance to the surface. We only need to tune this once because we will be coming back to the same height for each object.

In [4]:
disp = rm.tune_disparity_shift_dist(output='info')

Estimated a height of 0.245m
Disparity Shift: 141


### <u>Lets define our drop-off point
#### <b>Move the robot to the desired drop point

In [5]:
rm.align_gripper_with_z()
drop_waypoint = rm.get_joint_angles()
drop_waypoint = rm.normalize_wrist_angle(drop_waypoint)

### <u> Now lets find some objects!
Lets start by getting our point cloud and making some modifications:

In [7]:
# Move to our initial waypoint.
rm.set_joint_angles(pickup_waypoint) 
# Retrieve point cloud.
cloud = rm.get_point_cloud()
# Compress the point cloud with a voxel size of 3mm.
cloud_vg = rm.downsample_cloud(cloud,leaf_size=0.003)
# Remove the table.
cloud_vg_nt = rm.remove_planar_surface(cloud_vg)

Now lets segment objects pick the highest object, and generate a pose to pick it up.

In [11]:
# Segment objects with spreading segmentation algorithm.
object_clouds = rm.segment_cloud(cloud_vg_nt,search_radius=0.0033)

if len(object_clouds) > 0:
    print('Objects found: {}'.format(len(object_clouds)))

    # Sort object clouds by height.
    object_clouds_sorted = rm.sort_clouds_height(object_clouds)
    # Pick the highest cloud as our object.
    my_object = object_clouds_sorted[0]

    # Find the transformation matrix of this object representing position and orientation.
    # The transformation matrix will be positioned on the top of the object with the x axis
    # alligned with the principal axis of the object.
    object_transform = rm.get_object_transform(my_object, vertical=True)
    # Shift the transform down toward the table.
    
    ##CHANGE
    object_transform = rm.shift_transform_to_grasp(cloud_vg,object_transform)
    # Transform the object transform to a global frame relative to the robot base
    global_transform = rm.transform_transform(object_transform,camera_transform)

    object_width = rm.get_object_width(my_object) + 0.01
    
    # Convert the transformation matrix to a TCP pose.
    object_pose = rm.convert_transform_to_pose(global_transform)
    # translate the pose up to define another waypoint above the object
    object_pose_above = rm.translate_pose(object_pose,0,0,-0.1)
else:
    print("No objects found!")

Objects found: 2


### <u>Lets view the point cloud to make sure we like the proposed grasp

In [12]:
# Initialize a viewer object
view = rm.PC_Viewer()
view.add_cloud(cloud_vg,colorize=True,color=[100,100,100])
view.add_cloud(cloud_vg_nt)
for object_cloud in object_clouds:
    view.add_cloud(object_cloud,colorize=True)
view.add_axis(object_transform)

# We can also generate boxes representing the gripper given a transform and gripper width
gripper_boxes,finger_boxes = rm.get_gripper_boxes(object_transform,object_width)
view.add_gripper_boxes(gripper_boxes,finger_boxes)
view.show()

/home/nvidia/.local/lib/python3.5/site-packages/pythreejs/traits.py:177: UserWarning: 64-bit data types not supported for WebGL data, casting to 32-bit.
  warnings.warn('64-bit data types not supported for WebGL '


Renderer(camera=PerspectiveCamera(aspect=1.6, fov=90.0, position=(-0.04114200742896765, 0.12722675147906864, 0…

### <u>Now lets pick up the object and place it

In [13]:
rm.set_gripper_width(object_width)

# Set the torque low in case the gripper collides with something
rm.set_gripper_torque(15)

rm.movej(object_pose_above)
rm.movel(object_pose)

rm.set_gripper_torque(200)
time.sleep(0.5)
rm.close_gripper()

rm.movel(object_pose_above)
rm.set_joint_angles(drop_waypoint)

rm.open_gripper()

0